# IITM RL FINAL PROJECT

## Problem - bsuite 

This notebook uses an open source reinforcement learning benchmark known as bsuite. https://github.com/deepmind/bsuite

bsuite is a collection of carefully-designed experiments that investigate core capabilities of a reinforcement learning agent.

Your task is to use any reinforcement learning techniques at your disposal to get high scores on the environments specified.

**Note**: Since the course is on Reinforcement Learning,  please limit yourself to using traditional Reinforcement Learning algorithms, 

**Do not use deep reinforcement learning.**

# How to use this notebook? 📝

- This is a shared template and any edits you make here will not be saved. **You
should make a copy in your own drive**. Click the "File" menu (top-left), then "Save a Copy in Drive". You will be working in your copy however you like.

<p style="text-align: center"><img src="https://gitlab.aicrowd.com/aicrowd/assets/-/raw/master/notebook/aicrowd_notebook_submission_flow.png?inline=false" alt="notebook overview" style="width: 650px;"/></p>

- **Update the config parameters**. You can define the common variables here

Variable | Description
--- | ---
`AICROWD_RESULTS_DIR` | Path to write the output to.
`AICROWD_ASSETS_DIR` | In case your notebook needs additional files (like model weights, etc.,), you can add them to a directory and specify the path to the directory here (please specify relative path). The contents of this directory will be sent to AIcrowd for evaluation.
`AICROWD_API_KEY` | In order to submit your code to AIcrowd, you need to provide your account's API key. This key is available at https://www.aicrowd.com/participants/me

- **Installing packages**. Please use the [Install packages 🗃](#install-packages-) section to install the packages

In [ ]:
!pip install -q aicrowd-cli

     |████████████████████████████████| 61kB 5.2MB/s 
     |████████████████████████████████| 61kB 9.0MB/s 
     |████████████████████████████████| 174kB 40.3MB/s 
     |████████████████████████████████| 81kB 8.2MB/s 
     |████████████████████████████████| 61kB 8.7MB/s 
     |████████████████████████████████| 204kB 54.5MB/s 
     |████████████████████████████████| 71kB 9.8MB/s 
     |████████████████████████████████| 51kB 7.3MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


# AIcrowd Runtime Configuration 🧷

Get login API key from https://www.aicrowd.com/participants/me


In [ ]:
import os

AICROWD_RESULTS_DIR = os.getenv("OUTPUTS_DIR", "results")
os.environ["RESULTS_DIR"] = AICROWD_RESULTS_DIR
API_KEY = ""

In [ ]:
!aicrowd login --api-key $API_KEY

API Key valid
Saved API Key successfully!


# Install packages 🗃

Please add all pacakage installations in this section

In [ ]:
!pip install git+http://gitlab.aicrowd.com/nimishsantosh107/bsuite.git
!pip install tabulate
!pip install tqdm

## Add any other installations you need here

  Cloning http://gitlab.aicrowd.com/nimishsantosh107/bsuite.git to /tmp/pip-req-build-u1o9530n
  Running command git clone -q http://gitlab.aicrowd.com/nimishsantosh107/bsuite.git /tmp/pip-req-build-u1o9530n
  Created wheel for bsuite: filename=bsuite-0.3.5-cp37-none-any.whl size=252042 sha256=359552b6ce3d318b87095cd4360c460630927085e254d932e517a25c93248e8d
  Stored in directory: /tmp/pip-ephem-wheel-cache-uj1n1p45/wheels/61/ea/06/77c82c07765fb8608e50e6c66bc566fa6d113c725bc6937e7b
Successfully built bsuite


# Import packages

In [ ]:
import gym
import warnings

import numpy as np
import pandas as pd
import plotnine as gg
from tqdm.notebook import tqdm

import bsuite
from bsuite.aicrowd import environments
from bsuite.aicrowd.runner import Runner
from bsuite.aicrowd.analysis import Analyzer

pd.options.mode.chained_assignment = None
gg.theme_set(gg.theme_bw(base_size=16, base_family='serif'))
gg.theme_update(figure_size=(3, 1), panel_spacing_x=0.5, panel_spacing_y=0.5)
warnings.filterwarnings('ignore')

# **Agent Class**

You can modify the AGENT TEMPLATE below and implement the logic of your agent. Your agent must implement a few methods that will be called by the `Runner` class.
* `__init__` - put any initialization code here.
* `get_action` - takes in a `state` and returns an `action`.
* `learn` - takes in `(state, action, reward, next_state)`, implements the learning logic.
* `get_state` - takes in a raw `observation` directly from the env, discretizes it and returns a `state`.

In addition to these, you may implement other methods which can be called by the above methods.

Since there are multiple environments, you may need unique hyper parameters for each environment. Instantiate the agent while passing in the hyper parameters in a dictionary using the `agent_config` parameter so that each environment can use different hyper parameters for the agent while using a single `Agent` class for all of them.  You can use any names for the keys in the config dictionary.   

An example `RandomAgent` is given below.

In [ ]:
# *** YOU CAN EDIT THIS CELL ***
# AGENT TEMPLATE
class Agent:
    def __init__(self, agent_config=None):
      # declaring class variables which we will use in other class functions
        self.config = agent_config
        self.q_table = np.zeros((self.config['state_shape']+(self.config['action_space'],)))        
        self.num_ep_elapsed = 1

    def get_action(self, state):
        '''
        PARAMETERS  : 
            - state - discretized 'state'
        RETURNS     : 
            - action - 'action' to be taken
        '''   
        # epsilon greedy exploration
        if np.random.random() < self.get_epsilon() :
            return np.random.choice(self.config['action_space'])
        else :
            return np.argmax(self.q_table[state])
    
    def learn(self, state, action, reward, next_state,done):
        '''
        PARAMETERS  : 
            - state - discretized 'state'
            - action - 'action' performed in 'state'
            - reward - 'reward' received due to action taken
            - next_state - discretized 'next_state'
        RETURNS     : 
            - NIL
        '''
        max_future_q = np.max(self.q_table[next_state] )
        current_q_value= self.q_table[state][action]
        l_r = self.get_learning_rate()
        # Q learning update
        self.q_table[state] [action]= (1- l_r)* current_q_value + l_r * ( reward + self.config['discount_factor'] * max_future_q)
        if done :
            self.num_ep_elapsed += 1

    def get_epsilon(self):
      # calculating the epsilon for a given episode
        return self.config['epsilon_decay']/(self.config['epsilon_decay'] + self.num_ep_elapsed - 1)

    def get_learning_rate(self):
      # calculating the learning rate(step size) according to the function lr(t) = k/(k + t)
        return self.config['lr_decay']/(self.config['lr_decay'] + self.num_ep_elapsed)

    def get_state(self, observation):
        '''
        PARAMETERS  : 
            - observation - raw 'observation' from environment
        RETURNS     : 
            - state - discretized 'state' from raw 'observation'
        '''
        # computing the state for a given observation
        if self.config['env']=='catch' or self.config['env'] == 'catch_noise':
          # for catch env, we take the difference between columns as the state
            pos_1 = np.where(observation == 1)
            if len(np.where(observation[-1,:] == 1)[0]) == 2 :
              state = 5
            elif len(pos_1[0]) == 1 :
                state = 0
            else :
                state = pos_1[1][0] - pos_1[1][1]
            return tuple([state])
        elif self.config['env'] =='cartpole' :

            # make the states 
            in_boundary = None
            print(observation,self.config['high'])
            if self.config['high'][0] - 0.2 < observation[0][0] or self.config['low'][0] - 0.2 > observation[0][0] :
                in_boundary = 1
            else :
                in_boundary = 0

            pos_vel = int( observation[0][0] * observation[0][1] >0)
            theta_thetadot = int( observation[0][2] * observation[0][4] >0 )
            observation[0].append(0)
            observation[0].append(0)
            observation[0].append(0)

            observation = np.array(observation[0])
            discrete_win_size=(np.array(self.config["high"]) - self.config["low"])/(self.config["state_shape"]-np.ones(len(self.config["state_shape"])))
            dis_val  = (observation - self.config["low"])/discrete_win_size
            
            dis_obs = dis_val.astype(np.int)
            dis_obs=tuple(map(min,zip(self.config["state_shape"]-np.ones_like(self.config['state_shape']), tuple(map(max,zip(np.zeros_like(dis_obs), dis_obs))))))
            state = list(dis_obs)
            state[-3] = in_boundary
            state[-2] = pos_vel
            state[-1] = theta_thetadot
            return tuple(state)

        else:
          # for the other envs, we discretize the states into a grid and map each observation to one of the state
            observation = np.array(observation[0])
            discrete_win_size=(np.array(self.config["high"]) - self.config["low"])/(self.config["state_shape"]-np.ones(len(self.config["state_shape"])))
            dis_val  = (observation - self.config["low"])/discrete_win_size
            
            dis_obs = dis_val.astype(np.int)
            dis_obs=tuple(map(min,zip(self.config["state_shape"]-np.ones_like(self.config['state_shape']), tuple(map(max,zip(np.zeros_like(dis_obs), dis_obs))))))
            
            return dis_obs

In [ ]:
# *** YOU CAN EDIT THIS CELL ***
# EXAMPLE
class RandomAgent:
    def __init__(self, agent_config={}):
        self.config = agent_config

    def get_action(self, state):
        action = np.random.choice(2)
        return action
    
    def learn(self, state, action, reward, next_state,done):
        if ('BAR' in self.config):
            if (self.config['BAR']):
                self.config['FOO'] += 1

    def get_state(self, observation):
        state = observation
        return state

env1_config = {
    'FOO': 0.1,
    'BAR': True
}

env2_config = {
    'FOO': 0.2,
    'BAR': False
}

randomAgent1 = RandomAgent(agent_config=env1_config)
randomAgent2 = RandomAgent(agent_config=env2_config)

# **Playing with the Environment**

#### **Instantiating the environment** :
You can create an environment by calling the following function:  
`environments.load_env(ENV_ID)` - RETURNS: `env`  
where, ENV_ID can be ONE of the following:
* `environments.CATCH`
* `environments.CATCH_NOISE`
* `environments.CARTPOLE`
* `environments.CARTPOLE_NOISE`
* `environments.MOUNTAINCAR`
* `environments.MOUNTAINCAR_NOISE`

The `NOISE` environments add a scaled random noise to the `reward`.
<br/>

#### **Runnning the environment** :
There are certain methods required to run the environments. The interface is very similar to OpenAI Gym's interfaces. Fore more information, read the OpenAI documentation [here](https://gym.openai.com/docs/).

`env.reset()` - RETURNS: `observation`  
`env.step(action)`  - RETURNS: `(next_observation, reward, done, info[NOT USED])`

There are also a few useful properties within the environments:

* `env.action_space.n` - total number of possible actions. eg: if 'n' is 3, then the possible actions are `[0, 1, 2]`
* `env.observation_space.shape` -  the shape of the observation.
* `env.bsuite_num_episodes` -  the pre-specified number of episodes which will be run during evaluation (unique for each environment).

##### *ONLY IN CATCH / CATCH_NOISE*
* `env.observation_space.high` -  the upper limit for every index in the observation.
* `env.observation_space.low` -  the lower limit for every index of the observation.
<br/>


## **Environment Observation Space Limits:**

The limits for the observation space (minimum and maximum) for all the environments are given in the table below:

| Environments                        | Limits                                                                      |
|-------------------------------------|-----------------------------------------------------------------------------|
| CATCH <br/>  CATCH_NOISE            | MIN: use `env.observation_space.low` <br/> MAX: use `env.observation_space.high` |
| CARTPOLE <br/> CARTPOLE_NOISE       | MIN: `[-1. -5., -1., -1., -5., 0.]` <br/> MAX: `[ 1.,  5.,  1.,  1.,  5., 1.]` |
| MOUNTAINCAR <br/> MOUNTAINCAR_NOISE | MIN: `[-1.2, -0.07, 0.]` <br/> MAX: `[ 0.6,  0.07,  1.]`                                 |

[NOTE] Use this code cell to play around and get used to the environments. However, the `Runner` class below will be used to evaluate your agent.

In [ ]:
# *** YOU CAN EDIT THIS CELL ***
# DO NOT rename the config dictionaries as the evaluator references them. However, you may use any names for the keys in them.
catch_config = {'low':[-4],'high':[5],'action_space':3,'env':'catch','state_shape':(10,),'epsilon_decay':10,'lr_decay':100,'discount_factor':1}
catch_noise_config = {'low':[-4],'high':[5],'action_space':3,'env':'catch','state_shape':(10,),'epsilon_decay':10,'lr_decay':100,'discount_factor':1}
#cartpole_config = {'env':'cartpole','state_shape':(1,30,30,1,30,1),'action_space':3,'low':[-1., -5., -1., -1., -5., 0],'high':[ 1.1, 5.1, 1.1, 1.1, 5.2, 1],'epsilon_decay':25,'lr_decay':100,'discount_factor':1}
#cartpole_noise_config = {'env':'cartpole','state_shape':(1,30,30,1,30,1),'action_space':3,'low':[-1., -5., -1., -1., -5., 0],'high':[ 1.1, 5.1, 1.1, 1.1, 5.2, 1],'epsilon_decay':20,'lr_decay':100,'discount_factor':1}
cartpole_config = {'env':'cartpole','state_shape':(1,30,30,1,30,1,2,2,2),'action_space':3,'low':[-1., -5., -1., -1., -5., 0],'high':[ 1.1, 5.1, 1.1, 1.1, 5.2, 1],'epsilon_decay':50,'lr_decay':100,'discount_factor':1}
cartpole_noise_config = {'env':'cartpole','state_shape':(3,50,50,1,50,1),'action_space':3,'low':[-1., -5., -1., -1., -5., 0],'high':[ 1.1, 5.1, 1.1, 1.1, 5.2, 1],'epsilon_decay':50,'lr_decay':100,'discount_factor':1}
mountaincar_config = {'env':'mountaincar','state_shape':(10,10,1),'action_space':3,'low':[-1.2,-0.07,0],'high':[0.7,0.08,1],'epsilon_decay':1,'lr_decay':10,'discount_factor':0.95}
mountaincar_noise_config = {'env':'mountaincar','state_shape':(10,10,1),'action_space':3,'low':[-1.2,-0.07,0],'high':[0.7,0.08,1],'epsilon_decay':1,'lr_decay':10,'discount_factor':0.95}

In [ ]:
# *** YOU CAN EDIT THIS CELL ***
# TEST AREA
env = environments.load_env(environments.CARTPOLE)  # replace 'environments.CARTPOLE' with other environments

agent = Agent(agent_config=cartpole_config )                # replace with 'Agent()' to use your custom agent

NUM_EPISODES = 1000                            # replace with 'env.bsuite_num_episodes' to run for pre-specified number of episodes

for episode_n in tqdm(range(NUM_EPISODES)):
    done = False
    episode_reward = 0
    episode_moves = 0
 
    observation = env.reset()
    
    state = agent.get_state(observation)
   
    while not done:
        action = agent.get_action(state)
        next_observation, reward, done, _ = env.step(action)
        next_state = agent.get_state(next_observation)        
        agent.learn(state, action, reward, next_state,done)
        state = next_state
        episode_reward += reward
        episode_moves += 1
    if (((episode_n+1) % 50) == 0) : 
        print("EPISODE: ",episode_n+1,"\tREWARD: ",episode_reward,"\tEPISODE_LENGTH: ",episode_moves)
        

Loaded bsuite_id: cartpole/0.


  0%|          | 0/1000 [00:00<?, ?it/s]

EPISODE:  50 	REWARD:  84.0 	EPISODE_LENGTH:  85
EPISODE:  100 	REWARD:  141.0 	EPISODE_LENGTH:  142
EPISODE:  150 	REWARD:  1001.0 	EPISODE_LENGTH:  1001
EPISODE:  200 	REWARD:  1001.0 	EPISODE_LENGTH:  1001
EPISODE:  250 	REWARD:  1001.0 	EPISODE_LENGTH:  1001
EPISODE:  300 	REWARD:  1001.0 	EPISODE_LENGTH:  1001
EPISODE:  350 	REWARD:  1001.0 	EPISODE_LENGTH:  1001
EPISODE:  400 	REWARD:  1001.0 	EPISODE_LENGTH:  1001
EPISODE:  450 	REWARD:  1001.0 	EPISODE_LENGTH:  1001
EPISODE:  500 	REWARD:  1001.0 	EPISODE_LENGTH:  1001
EPISODE:  550 	REWARD:  232.0 	EPISODE_LENGTH:  233
EPISODE:  600 	REWARD:  1001.0 	EPISODE_LENGTH:  1001
EPISODE:  650 	REWARD:  1001.0 	EPISODE_LENGTH:  1001
EPISODE:  700 	REWARD:  1001.0 	EPISODE_LENGTH:  1001
EPISODE:  750 	REWARD:  1001.0 	EPISODE_LENGTH:  1001
EPISODE:  800 	REWARD:  1001.0 	EPISODE_LENGTH:  1001
EPISODE:  850 	REWARD:  1001.0 	EPISODE_LENGTH:  1001
EPISODE:  900 	REWARD:  1001.0 	EPISODE_LENGTH:  1001
EPISODE:  950 	REWARD:  1001.0 	EPISO

## Point to the Agent Class you'll use for the final score

In [ ]:
RLAgent = Agent

# **Evaluating the Agent on all the Environments**

* The following cells will take care of running your agent on each environment and aggregating the results in csv files. In each of the following cells, the `agent_config` parameter is already set to use the corresponding config dictionary for that environment. DO NOT EDIT THIS.
* Feel free to modify the `LOG_INTERVAL` parameter to change the interval between episodes for logging.  
* Please do not modify any other contents in each of the cells.  

In [ ]:
LOG_INTERVAL = 100

In [ ]:
runner = Runner(
    agent = RLAgent(agent_config=catch_config),
    env_id = environments.CATCH,
    log_interval = LOG_INTERVAL,
)
runner.play_episodes()

Loaded bsuite_id: catch/0.
Logging results to CSV file for each bsuite_id in results.


  0%|          | 0/10000 [00:00<?, ?it/s]

EPISODE:  100 	REWARD:  -1.0 	MEAN_REWARD:  0.44 	EPISODE_LENGTH:  9
EPISODE:  200 	REWARD:  1.0 	MEAN_REWARD:  0.98 	EPISODE_LENGTH:  9
EPISODE:  300 	REWARD:  1.0 	MEAN_REWARD:  0.88 	EPISODE_LENGTH:  9
EPISODE:  400 	REWARD:  1.0 	MEAN_REWARD:  1.0 	EPISODE_LENGTH:  9
EPISODE:  500 	REWARD:  1.0 	MEAN_REWARD:  0.98 	EPISODE_LENGTH:  9
EPISODE:  600 	REWARD:  1.0 	MEAN_REWARD:  1.0 	EPISODE_LENGTH:  9
EPISODE:  700 	REWARD:  1.0 	MEAN_REWARD:  1.0 	EPISODE_LENGTH:  9
EPISODE:  800 	REWARD:  1.0 	MEAN_REWARD:  0.94 	EPISODE_LENGTH:  9
EPISODE:  900 	REWARD:  1.0 	MEAN_REWARD:  1.0 	EPISODE_LENGTH:  9
EPISODE:  1000 	REWARD:  1.0 	MEAN_REWARD:  0.96 	EPISODE_LENGTH:  9
EPISODE:  1100 	REWARD:  1.0 	MEAN_REWARD:  1.0 	EPISODE_LENGTH:  9
EPISODE:  1200 	REWARD:  1.0 	MEAN_REWARD:  1.0 	EPISODE_LENGTH:  9
EPISODE:  1300 	REWARD:  1.0 	MEAN_REWARD:  1.0 	EPISODE_LENGTH:  9
EPISODE:  1400 	REWARD:  1.0 	MEAN_REWARD:  1.0 	EPISODE_LENGTH:  9
EPISODE:  1500 	REWARD:  1.0 	MEAN_REWARD:  0.98 	

In [ ]:
runner = Runner(
    agent = RLAgent(agent_config=catch_noise_config),
    env_id = environments.CATCH_NOISE,
    log_interval = LOG_INTERVAL
)
runner.play_episodes()

Loaded bsuite_id: catch_noise/1.
Logging results to CSV file for each bsuite_id in results.


  0%|          | 0/10000 [00:00<?, ?it/s]

EPISODE:  100 	REWARD:  -0.4319180485080193 	MEAN_REWARD:  0.59 	EPISODE_LENGTH:  9
EPISODE:  200 	REWARD:  2.888194910660849 	MEAN_REWARD:  1.04 	EPISODE_LENGTH:  9
EPISODE:  300 	REWARD:  0.7058772924254098 	MEAN_REWARD:  0.91 	EPISODE_LENGTH:  9
EPISODE:  400 	REWARD:  0.5749511984405982 	MEAN_REWARD:  0.94 	EPISODE_LENGTH:  9
EPISODE:  500 	REWARD:  1.8165310035839968 	MEAN_REWARD:  1.07 	EPISODE_LENGTH:  9
EPISODE:  600 	REWARD:  0.39936420173707576 	MEAN_REWARD:  1.07 	EPISODE_LENGTH:  9
EPISODE:  700 	REWARD:  1.3884776851441103 	MEAN_REWARD:  0.91 	EPISODE_LENGTH:  9
EPISODE:  800 	REWARD:  1.8585504709523746 	MEAN_REWARD:  0.92 	EPISODE_LENGTH:  9
EPISODE:  900 	REWARD:  1.362278430457311 	MEAN_REWARD:  0.99 	EPISODE_LENGTH:  9
EPISODE:  1000 	REWARD:  1.829857544425355 	MEAN_REWARD:  1.01 	EPISODE_LENGTH:  9
EPISODE:  1100 	REWARD:  -0.37396752528475785 	MEAN_REWARD:  1.01 	EPISODE_LENGTH:  9
EPISODE:  1200 	REWARD:  0.9410664532715053 	MEAN_REWARD:  1.02 	EPISODE_LENGTH:  9


In [ ]:
runner = Runner(
    agent = RLAgent(agent_config=cartpole_config),
    env_id = environments.CARTPOLE,
    log_interval = LOG_INTERVAL
)
runner.play_episodes()

Loaded bsuite_id: cartpole/0.
Logging results to CSV file for each bsuite_id in results.


  0%|          | 0/1000 [00:00<?, ?it/s]

[[0.00162712 0.00717298 0.01027616 0.9999472  0.00448832 0.        ]] [1.1, 5.1, 1.1, 1.1, 5.2, 1]


AttributeError: ignored

In [ ]:
runner = Runner(
    agent = RLAgent(agent_config=cartpole_noise_config),
    env_id = environments.CARTPOLE_NOISE,
    log_interval = LOG_INTERVAL
)
runner.play_episodes()

Loaded bsuite_id: cartpole_noise/1.
Logging results to CSV file for each bsuite_id in results.


  0%|          | 0/1000 [00:00<?, ?it/s]

EPISODE:  100 	REWARD:  231.352157302122 	MEAN_REWARD:  142.34 	EPISODE_LENGTH:  231
EPISODE:  200 	REWARD:  210.82923120035548 	MEAN_REWARD:  303.31 	EPISODE_LENGTH:  210
EPISODE:  300 	REWARD:  1008.5102108507523 	MEAN_REWARD:  643.29 	EPISODE_LENGTH:  1001
EPISODE:  400 	REWARD:  995.4763147515752 	MEAN_REWARD:  912.83 	EPISODE_LENGTH:  1001
EPISODE:  500 	REWARD:  945.8247368041087 	MEAN_REWARD:  868.28 	EPISODE_LENGTH:  942
EPISODE:  600 	REWARD:  773.4217301178663 	MEAN_REWARD:  956.91 	EPISODE_LENGTH:  777
EPISODE:  700 	REWARD:  988.2567534134251 	MEAN_REWARD:  871.77 	EPISODE_LENGTH:  1001
EPISODE:  800 	REWARD:  427.5943116676481 	MEAN_REWARD:  976.6 	EPISODE_LENGTH:  433
EPISODE:  900 	REWARD:  988.2062821633898 	MEAN_REWARD:  921.54 	EPISODE_LENGTH:  1001
EPISODE:  1000 	REWARD:  551.1812400178924 	MEAN_REWARD:  952.02 	EPISODE_LENGTH:  551


In [ ]:
runner = Runner(
    agent = RLAgent(agent_config=mountaincar_config),
    env_id = environments.MOUNTAINCAR,
    log_interval = LOG_INTERVAL
)
runner.play_episodes()

Loaded bsuite_id: mountain_car/0.
Logging results to CSV file for each bsuite_id in results.


  0%|          | 0/1000 [00:00<?, ?it/s]

EPISODE:  100 	REWARD:  -191.0 	MEAN_REWARD:  -477.22 	EPISODE_LENGTH:  191
EPISODE:  200 	REWARD:  -266.0 	MEAN_REWARD:  -280.07 	EPISODE_LENGTH:  266
EPISODE:  300 	REWARD:  -170.0 	MEAN_REWARD:  -252.13 	EPISODE_LENGTH:  170
EPISODE:  400 	REWARD:  -175.0 	MEAN_REWARD:  -205.9 	EPISODE_LENGTH:  175
EPISODE:  500 	REWARD:  -248.0 	MEAN_REWARD:  -181.11 	EPISODE_LENGTH:  248
EPISODE:  600 	REWARD:  -159.0 	MEAN_REWARD:  -225.09 	EPISODE_LENGTH:  159
EPISODE:  700 	REWARD:  -159.0 	MEAN_REWARD:  -172.19 	EPISODE_LENGTH:  159
EPISODE:  800 	REWARD:  -139.0 	MEAN_REWARD:  -200.28 	EPISODE_LENGTH:  139
EPISODE:  900 	REWARD:  -153.0 	MEAN_REWARD:  -193.58 	EPISODE_LENGTH:  153
EPISODE:  1000 	REWARD:  -168.0 	MEAN_REWARD:  -207.89 	EPISODE_LENGTH:  168


In [ ]:
runner = Runner(
    agent = RLAgent(agent_config=mountaincar_noise_config),
    env_id = environments.MOUNTAINCAR_NOISE,
    log_interval = LOG_INTERVAL
)
runner.play_episodes()

Loaded bsuite_id: mountain_car_noise/1.
Logging results to CSV file for each bsuite_id in results.


  0%|          | 0/1000 [00:00<?, ?it/s]

EPISODE:  100 	REWARD:  -215.82493296638668 	MEAN_REWARD:  -412.94 	EPISODE_LENGTH:  213
EPISODE:  200 	REWARD:  -146.43883197918012 	MEAN_REWARD:  -216.51 	EPISODE_LENGTH:  148
EPISODE:  300 	REWARD:  -302.6861930747661 	MEAN_REWARD:  -189.61 	EPISODE_LENGTH:  311
EPISODE:  400 	REWARD:  -142.4670140369222 	MEAN_REWARD:  -202.86 	EPISODE_LENGTH:  142
EPISODE:  500 	REWARD:  -150.246092435592 	MEAN_REWARD:  -154.24 	EPISODE_LENGTH:  151
EPISODE:  600 	REWARD:  -163.1620010565664 	MEAN_REWARD:  -155.81 	EPISODE_LENGTH:  163
EPISODE:  700 	REWARD:  -150.1282561295434 	MEAN_REWARD:  -168.09 	EPISODE_LENGTH:  146
EPISODE:  800 	REWARD:  -149.3528434448176 	MEAN_REWARD:  -159.53 	EPISODE_LENGTH:  149
EPISODE:  900 	REWARD:  -144.36874886148814 	MEAN_REWARD:  -161.04 	EPISODE_LENGTH:  146
EPISODE:  1000 	REWARD:  -149.89819891657865 	MEAN_REWARD:  -153.04 	EPISODE_LENGTH:  151


# **Analysis & Result**

The following cells will show the score of the agent on each environment. The same scoring method will be used to evaluate your agent on a set of test environments.

In [ ]:
# *** PLEASE DONT EDIT THE CONTENTS OF THIS CELL ***
analyzer = Analyzer(os.environ.get('RESULTS_DIR'))
analyzer.print_scores()

╒════════════════════╤══════════╕
│ ENVIRONMENT        │    SCORE │
╞════════════════════╪══════════╡
│ catch              │ 0.993125 │
├────────────────────┼──────────┤
│ catch_noise        │ 0.993875 │
├────────────────────┼──────────┤
│ cartpole           │ 0.873785 │
├────────────────────┼──────────┤
│ cartpole_noise     │ 0.877328 │
├────────────────────┼──────────┤
│ mountain_car       │ 0.860454 │
├────────────────────┼──────────┤
│ mountain_car_noise │ 0.90245  │
╘════════════════════╧══════════╛


In [ ]:
# If you want a object to get the scores
analyzer.get_scores()

{'cartpole': 0.8737855,
 'cartpole_noise': 0.8773275,
 'catch': 0.993125,
 'catch_noise': 0.993875,
 'mountain_car': 0.8604539999999999,
 'mountain_car_noise': 0.9024500000000001}